In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from ebany_research.llm_lora.changed_mistral import (
    LinearLora,
    ChangedMistralForCausalLM,
)
from transformers import AutoTokenizer, AutoConfig
import torch

model_name = "Open-Orca/Mistral-7B-OpenOrca"
lora_model_name = "ebany_research/llm_lora/models/"
lora_model_name += "52[6c_11c_14_17c_20c_22c_25c_26][6_10_11_14_17_20_22_25_26_30][6c_10c_11c_14c_17c_20_22c_25c_26c_30c]"

config = AutoConfig.from_pretrained(lora_model_name)
student_model = ChangedMistralForCausalLM.from_pretrained(
    lora_model_name,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
)
student_model = student_model.eval()
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:21<00:00,  7.20s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
with torch.no_grad():
    chat = [
        {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer."},
        # {
        #     "role": "user",
        #     "content": """Please add punctuation to this: The cause of the explosion has never been clear the affected area was completely uninhabited""",
        # },
        # {"role": "assistant", "content": dataset_item["response"]},
    ]
    inputs_text = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        # add_generation_prompt=True,
    )
    # print(inputs)
    inputs = tokenizer(
        inputs_text,
        return_tensors="pt",
        truncation=True,
        # max_length=4096,
        max_length=2048,
    ).to(student_model.device)

    outputs = student_model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        # top_k=50,
        # top_p=0.95,
    )
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=False)
    print(outputs[0])

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<s><|im_start|> system
You are an AI assistant. You will be given a task. You must generate a detailed and long answer.<|im_end|> 

The task is to generate a response to the statement "I am a professional AI assistant and I have been assigned to this task.".

The task involves generating a response to the statement "I am a professional AI assistant and I have been assigned to this task.".

The task involves generating a response to the statement "I am a professional AI assistant and I have been assigned to this task.".

The task involves generating a response to the statement "I am a professional AI assistant and I have been assigned to this task.".

The task involves generating a response to the statement "I am a professional AI assistant and I have been assigned to this task."<|im_start|> response is as follows:

"I am a professional AI assistant and I have been assigned to this task."

"I am a professional AI assistant and I have been assigned to this task."

"I am a professional AI

The A in COATS stands for Cadet Organations Administration and Training Service. It is a cadet organization that provides training, safety, supervision and administration for cadet members in the Canadian Cadet Movement. The Cadet Instructors Cadle (CIC) branch of the CAF is a part of the Cadet Organations Administration and Training Service. The Cadet Instructors Supplementary Staff List (CISS List) is a list of cadet instructors who may be held in anticipation of their employment in the same manner as other cadet instructors are held as members of the Supplementary Reserve.

In [3]:
tokenizer(
    ["hello world hello world hello worldhello world", "hello", ],
    # return_tensors="pt",
    truncation=True,
    # max_length=4096,
    max_length=2048,
    padding=True,
)

{'input_ids': [[1, 6312, 28709, 1526, 6312, 28709, 1526, 6312, 28709, 1526, 21558, 1526], [32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 1, 6312, 28709]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]}

In [11]:
chat = [
    {
        "role": "system",
        "content": "You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.",
    },
    {
        "role": "user",
        "content": """David has three sisters. Each of them has one brother. How many brothers does David have?""",
    },
    # {"role": "assistant", "content": "The Confederation of British Industry (CBI) and British Chambers of Commerce have called for market stability and political clarity following the UK's decision to leave the EU. Bank of England Governor Mark Carney has promised support for financial markets, announcing the Bank's preparedness to provide £250bn to support markets during any periods of instability. Business leaders have expressed concerns over resulting uncertainty slowing economic activity, and companies are seeking immediate clarity on a timetable to leave the EU. Meanwhile, union leaders' priority is to protect jobs and living standards of the UK's working population."},
]
inputs = tokenizer.apply_chat_template(
    chat,
    tokenize=False,
    # add_generation_prompt=True,
)
inputs

'<|im_start|>system\nYou are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.<|im_end|>\n<|im_start|>user\nDavid has three sisters. Each of them has one brother. How many brothers does David have?<|im_end|>\n'